<a href="https://colab.research.google.com/github/jeanbouteiller-ds/tennis_prediction/blob/main/workflow_notebooks/merge_tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import subprocess
import sys

def install_package(package_name):
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package_name])
        print(f"Successfully installed {package_name}")
    except subprocess.CalledProcessError:
        print(f"Failed to install {package_name}") # Install the PyGitHub library

install_package("requests")
install_package("PyGitHub")
install_package("pandas")
install_package("tqdm")
install_package("mumpy")

Failed to install mumpy


In [2]:
import requests
import io
import importlib.util
import sys
import nbformat
import pandas as pd
import math


# Define the URL of the nb1 notebook on GitHub
github_url = "https://raw.githubusercontent.com/jeanbouteiller-ds/tennis_prediction/main/functions/functions_fixed_table.ipynb"

# Download the notebook as a raw .ipynb file
response = requests.get(github_url)

if response.status_code == 200:
  notebook_content = response.text

  # Parse the notebook content
  notebook = nbformat.reads(notebook_content, as_version=4)

  # Now you can execute the cells in the notebook
  for cell in notebook.cells:
      if cell.cell_type == 'code':
          exec(cell.source)

In [3]:
df_matchs=get_file_content('data/clean_datasets/df_matchs_clean.csv')
df_ranking=get_file_content('data/clean_datasets/df_ranking_clean.csv')
df_betting=get_file_content('data/clean_datasets/df_betting_clean.csv')
df_fixed_table=get_file_content('data/clean_datasets/df_fixed_table_clean.csv')
df_tournaments=get_file_content('data/clean_datasets/df_tournaments_clean.csv')

# Add Player Ranking

In [4]:
import pandas as pd

# Convert date columns to datetime if they are not already
df_matchs['tournament_date'] = pd.to_datetime(df_matchs['tournament_date'])
df_ranking['date'] = pd.to_datetime(df_ranking['date'])


# Merge based on the closest date before tournament_date to find_winner_raking
df_ranking['winner_name_clean_3first'] = df_ranking['player_name_clean_3first']
merged_data_winner = pd.merge_asof(df_matchs.sort_values('tournament_date'),
                            df_ranking.sort_values('date'),
                            by=['winner_name_clean_3first'],
                            left_on='tournament_date',
                            right_on='date',
                            direction='backward')

merged_data_winner.rename(columns={'ranking': 'ranking_winner'}, inplace=True)
final_df=merged_data_winner.drop(["player_name","date","player_name_clean_3first"],axis=1)

# Merge based on the closest date before tournament_date to fin loser_ranking
df_ranking=df_ranking.drop('winner_name_clean_3first',axis=1)
df_ranking['loser_name_clean_3first'] = df_ranking['player_name_clean_3first']
merged_data_loser = pd.merge_asof(final_df.sort_values('tournament_date'),
                            df_ranking.sort_values('date'),
                            by=['loser_name_clean_3first'],
                            left_on='tournament_date',
                            right_on='date',
                            direction='backward')

# Rename the 'ranking' column from df_ranking to 'ranking_winner'
merged_data_loser.rename(columns={'ranking': 'ranking_loser'}, inplace=True)

# Now, merged_data contains the desired dataset
df_all_with_ranking=merged_data_loser.drop(["tournament_name","player_name","date","player_name_clean_3first"],axis=1)

# Add odds

The code below takes a bit of time to run :)

In [ ]:
import pandas as pd


# Merge based on the conditions
merged_data = df_all_with_ranking.merge(
    df_betting[['unique_tournament_id','clean_winner_only_name_full','clean_loser_only_name_full',
                'Final_odd_winner','Final_odd_loser']],
    how='left',
    left_on=['unique_tournament_id'],
    right_on=['unique_tournament_id']
)

merged_data["unique_game_id"]=merged_data['winner_name']+"//"+merged_data['loser_name']+"//"+merged_data["unique_tournament_id"]

condition1 = merged_data.apply(lambda row: str(row['clean_loser_only_name_full']) in str(row['loser_name']), axis=1)
condition2 = merged_data.apply(lambda row: str(row['clean_winner_only_name_full']) in str(row['winner_name']), axis=1)

df_ranking_odds = merged_data[condition1 & condition2].drop_duplicates("unique_game_id")
no_odds_games=merged_data[(~merged_data['unique_game_id'].isin(df_ranking_odds['unique_game_id'].unique()))].drop_duplicates("unique_game_id")
no_odds_games['clean_winner_only_name_full']=math.nan
no_odds_games['clean_loser_only_name_full']=math.nan
no_odds_games['Final_odd_winner']=math.nan
no_odds_games['Final_odd_loser']=math.nan
df_ranking_odds= pd.concat([df_ranking_odds,no_odds_games])
df_ranking_odds
# df_ranking_odds = pd.concat(merged_data)
# Now, merged_data contains the rows where all three conditions are met (based on inclusion)


# Add Df Fixed Data

In [ ]:
import pandas as pd

# Merge based on winner_name_clean_3first
merged_data = df_ranking_odds.merge(
    df_fixed_table,  # Replaced 'df_fixed' with 'df_fixed_table'
    left_on='winner_name_clean_3first',
    right_on='player_name_clean_3first',
    how='left'
)

# Rename columns from df_fixed_table with 'winner:' prefix
for col in df_fixed_table.columns:
    merged_data.rename(columns={col: 'winner: ' + col}, inplace=True)

# Drop the original 'player_name_clean_3first' column
merged_data.drop('winner: player_name_clean_3first', axis=1, inplace=True)

# Merge again based on loser_name_clean_3first
merged_data = merged_data.merge(
    df_fixed_table,  # Replaced 'df_fixed' with 'df_fixed_table'
    left_on='loser_name_clean_3first',
    right_on='player_name_clean_3first',
    how='left'
)

# Rename columns from df_fixed_table with 'loser:' prefix
for col in df_fixed_table.columns:
    merged_data.rename(columns={col: 'loser: ' + col}, inplace=True)

# Drop the original 'player_name_clean_3first' columns
merged_data.drop(['loser: player_name_clean_3first', 'winner_name_clean_3first', 'loser_name_clean_3first',
                  ], axis=1, inplace=True)

# Now, merged_data contains the desired columns with 'winner:' and 'loser:' prefixes
df_ranking_odds_fixed=merged_data.copy()

# Add tournaments Data

In [ ]:
df_ranking_odds_fixed_tournament=df_ranking_odds_fixed.merge(df_tournaments,left_on="unique_tournament_id",right_on="unique_tournament_id").drop("tournament_date_y",axis=1).rename({"tournament_date_x":"tournament_date"},axis=1)


# Create the binary_transformation with train,validation & test separation

In [ ]:
import numpy as np

np.random.seed(seed=42)
df_ranking_odds_fixed_tournament['winner'] = np.random.randint(2, size=len(df_ranking_odds_fixed_tournament))
add_file_github('data/clean_datasets/df_all_merged.csv',df_ranking_odds_fixed_tournament.to_csv(index=False))

df=df_ranking_odds_fixed_tournament.copy()[['tournament_date','unique_game_id']]

# Convert 'tournament_date' to datetime if it's not already in datetime format
df['tournament_date'] = pd.to_datetime(df['tournament_date'])

# Split the dataset based on date ranges
train = df[df['tournament_date'].dt.year <= 2017].drop(['tournament_date'],axis=1).drop_duplicates("unique_game_id")
validation = df[(df['tournament_date'].dt.year >= 2018) & (df['tournament_date'].dt.year <= 2020)].drop(['tournament_date'],axis=1).drop_duplicates("unique_game_id")
test = df[(df['tournament_date'].dt.year >= 2021) & (df['tournament_date'].dt.year <= 2023)].drop(['tournament_date'],axis=1).drop_duplicates("unique_game_id")


In [ ]:
add_file_github('data/ml_datasets/train_games_ids.csv',train.to_csv(index=False))
add_file_github('data/ml_datasets/validation_games_ids.csv',validation.to_csv(index=False))
add_file_github('data/ml_datasets/test_games_ids.csv',test.to_csv(index=False))